In [333]:
from dataclasses import dataclass
import numpy as np

In [334]:
@dataclass
class Point:
    x: int
    y: int
    wire1: bool = False
    wire2: bool = False
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    #Manhattan distance
    def distance(self, other: Point) -> int:
        return abs(self.x - other.x) + abs(self.y - other.y)
    
    def both_wires(self):
        return wire1 and wire2
    
@dataclass
class Instruction:
    direction: str
    amount: int
        
    def get_next_coords(self, x: int, y: int):
        if self.direction == 'R':
            coords = [(i, y) for i in range(x+1, x+self.amount+1)]
        if self.direction == 'L':
            coords = [(i, y) for i in range(x-self.amount, x)]
            coords.reverse()
        if self.direction == 'U':
            coords = [(x, i) for i in range(y+1, y+self.amount+1)]
        if self.direction == 'D':
            coords = [(x, i) for i in range(y-self.amount, y)]
            coords.reverse()
        return coords

In [335]:
def parse(element:str):
    # Task-specific element parsing here
    direction, *amount = element
    return Instruction(direction, int(''.join(amount)))

In [336]:
filename = 'input.txt'
#filename = 'test2.txt'
with open(filename) as f:
    line1, line2 = f.readlines()
    wire1  = [parse(element) for element in line1.replace('/n', '').split(',')]
    wire2  = [parse(element) for element in line2.replace('/n', '').split(',')]

# Part 1 

## Attempt 1

```
# Build map of x -> y -> value
current_x = current_y = 0
grid = [Point(current_x, current_y, True, True)]
# Wire1
for instruction in wire1:
    coords = instruction.get_next_coords(current_x, current_y)
    print((current_x, current_y))
    print(instruction)
    print(coords)
    current_x, current_y = coords[-1]
    for x, y in coords:
        point = Point(x, y, True)
        if point not in grid:
            grid.append(point)
current_x = current_y = 0
#Wire2
for instruction in wire2:
    coords = instruction.get_next_coords(current_x, current_y)
    current_x, current_y = coords[-1]
    for x, y in coords:
        print(grid)
        point = Point(x, y, False, True)
        if point in grid:
            grid.remove(point)
            print('found a cross')
            point.wire1 = point.wire2 = True
            print(point)
            grid.append(point)
crossed_wires = [p for p in grid if p != Point(0,0) and p.wire1 and p.wire2]
```

## Attempt 2

In [337]:
# Build map of x -> y -> value
current_x = current_y = 0
grid = {'0,0': {'wire1': True, 'wire2': True}}
# Wire1
for instruction in wire1:
    coords = instruction.get_next_coords(current_x, current_y)
    current_x, current_y = coords[-1]
    for x, y in coords:
        point = f'{x},{y}'
        if point not in grid:
            grid[point] = {'wire1': True, 'wire2': False}
current_x = current_y = 0
#Wire2
for instruction in wire2:
    coords = instruction.get_next_coords(current_x, current_y)
    current_x, current_y = coords[-1]
    for x, y in coords:
        point = f'{x},{y}'
        if point in grid and grid[point]['wire1'] and not grid[point]['wire2']:
            grid[point]['wire2'] = True
        else:
            grid[point] = {'wire1': False, 'wire2': True}
crossed_wires = [Point(*[int(i) for i in p.split(',')], True, True) for p, wires in grid.items() 
                 if p != '0,0' and wires['wire1'] and wires['wire2']]

In [338]:
centre = Point(0,0)
distances = [(p, p.distance(centre)) for p in crossed_wires]
print(min([d for _, d in distances]))

2129


# Part 2

In [339]:
# Build map of x -> y -> value
current_x = current_y = 0
signal = 0
grid = {'0,0': {'wire1': True, 'wire2': True, 'signal1': 0, 'signal2': 0}}
# Wire1
for instruction in wire1:
    coords = instruction.get_next_coords(current_x, current_y)
    current_x, current_y = coords[-1]
    for x, y in coords:
        point = f'{x},{y}'
        signal += 1
        if point not in grid:
            grid[point] = {'wire1': True, 'wire2': False, 'signal1': signal, 'signal2': 0}
current_x = current_y = 0
signal = 0
#Wire2
for instruction in wire2:
    coords = instruction.get_next_coords(current_x, current_y)
    current_x, current_y = coords[-1]
    for x, y in coords:
        point = f'{x},{y}'
        signal += 1
        if point in grid and grid[point]['wire1'] and not grid[point]['wire2']:
            grid[point]['wire2'] = True
            grid[point]['signal2'] = signal
        else:
            grid[point] = {'wire1': False, 'wire2': True, 'signal1': 0, 'signal2': signal}
crossed_wires = [(p, wires['signal1'] + wires['signal2']) for p, wires in grid.items() 
                 if p != '0,0' and wires['wire1'] and wires['wire2']]

In [340]:
print(min([d for _, d in crossed_wires]))

134662
